## Problem Statement:

** Explore the datasets and develop a model to predict customer churn over time. **

**By:** Yusuf Firoz

## Step 1: Data PreProcessing

#### Import required libraries

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from time import time
import pickle

#### Import all the CSV files

In [2]:
boxes = pd.read_csv('boxes.csv')
cancels = pd.read_csv('cancels.csv')
errors = pd.read_csv('errors.csv')
pauses = pd.read_csv('pauses.csv')

## Pauses Table

** Data Explration **
* This table has 3 columns named **'subscription_id'**,	**'pause_start'**',	**'pause_end'**
* **'pause_start'** and **'pause_end'** are given as string which needs to be converted to Timestamp 

** Data Cleaning **
* '2017-01-06' value in  **'pause_end'** is mistyped as '2016-01-06'. Correction is done.

** Feature Engineering **
* Duration between the first and last pause will be calculated.
* Total No of pauses between that duration will be calculated.

** Feature Insights **
* **First_pause_start**: When Pause occured for the first time.
* **Last_pause_end**: When Pause occured for the last time.
* **Total_pause_duration**: Duration between the first and last pause.
* **Total_no_of_Pauses**: Total no of pauses between that duration.

* Divide the Total_pause_duration by Total_no_of_Pauses to maintain the scale.

In [3]:
pauses.head()

,subscription_id,pause_start,pause_end
0,1000001.0,2015-02-09,2015-02-15
1,1000001.0,2014-11-24,2014-11-30
2,1000001.0,2015-02-02,2015-02-08
3,1000001.0,2015-02-16,2015-02-22
4,1000001.0,2015-01-12,2015-01-18


In [ ]:
#Convert 'pause_start' and 'pause_end' to Timestamp.
pauses.iloc[:,1] = pd.to_datetime(pauses.iloc[:,1], format='%Y-%m-%d')
pauses.iloc[:,2] = pd.to_datetime(pauses.iloc[:,2], format='%Y-%m-%d')

#Add columns named 'Total_pause_duration' of each subscriber. 
pauses['Total_pause_duration'] = pauses['pause_end'].subtract(pauses['pause_start']) 

# '2017-01-06' is mistyped as '2016-01-06'
pauses['pause_end'][pauses.Total_pause_duration == '-360 days'] = '2017-01-06'

#Convert 'pause_end to Timestamp again.
pauses.iloc[:,2] = pd.to_datetime(pauses.iloc[:,2], format='%Y-%m-%d')

#Calculate 'Total_pause_duration' again
pauses['Total_pause_duration'] = pauses['pause_end'].subtract(pauses['pause_start']) 


#We can take standard Deviation or average in place of min and max of 'Total_pause_duration'. It might get us better result
pauses_groupby = pauses.groupby('subscription_id').agg({'pause_start':['min', 'size'], 'pause_end':['max'], 'Total_pause_duration':['sum']}).reset_index() 

#Naming columns
pauses_groupby.columns = ['subscription_id','First_pause_start','Total_no_of_Pauses','Last_pause_end', 'Total_pause_duration']



In [5]:
pauses_groupby.head()

,subscription_id,First_pause_start,Total_no_of_Pauses,Last_pause_end,Total_pause_duration
0,1000001.0,2014-11-24,10,2015-03-01,60 days
1,1000002.0,2012-12-17,1,2012-12-23,6 days
2,1000003.0,2012-11-12,15,2013-05-19,90 days
3,1000004.0,2012-12-10,15,2013-10-20,90 days
4,1000005.0,2013-03-25,1,2013-03-31,6 days


** Pickle file for Pauses** 

In [6]:
#Create pickle file for Pauses datset
with open('pauses.pickle', 'wb') as f:
    pickle.dump(pauses_groupby, f)

# Errors Tables

** Data Explration **
* This table has 6 columns named **'subscription_id'**,	**'reported_date'**', **'hellofresh_week_where_error_happened'**, **'compensation_type'**, **'compensation_amount'**
* **'reported_date'** is given as string which needs to be converted to Timestamp 



** Data Cleaning **
* In **'compensation_type'**, one value is given in two ways, i.e. 'full_refund' and 'full refund'. Remove this ambiguity.

** Feature Engineering **
* Convert the **compensation_type** and **compensation_amount** to numerical features using one-hot encoding scheme. It is  creating dummy variables for all the classes of categorical featues.


** Feature Insights **
* **first_reported_date**: When was the first conflict happened.
* **last_reported_date**: When was the last conflict happened.
* **no_of_conflict**: Total no of conflicts or issues occured with a subscriber.
* **'total_CA', 'max_CA','min_CA','mean_CA'**: Dummy variables for **compensation_amount**
* **''credit', 'full_refund','none','partial_refund','refund','sorry'**: Dummy variables for **compensation_type**




In [7]:
errors.head(5)

,subscription_id,reported_date,hellofresh_week_where_error_happened,compensation_type,compensation_amount
0,1000001,2014-12-05,2014-W49,credit,10.0
1,1000010,2015-06-09,2015-W31,credit,25.0
2,1000010,2015-04-02,2015-W16,credit,25.0
3,1000010,2016-02-17,2016-W07,full_refund,69.0
4,1000012,2015-05-28,2015-W22,credit,34.5


In [8]:
#Convert to timestamp
errors.iloc[:,1] = pd.to_datetime(errors.iloc[:,1], format='%Y-%m-%d')

#Remove ambiguity of 'full_refund' and 'full refund'
errors['compensation_type'] = errors['compensation_type'].replace(['full refund'], 'full_refund')

#creating  dummy variables for categorical attribute 'errors_compensation_type'. We can remove one column to avoid collinearity.
errors_compensation_type = pd.get_dummies(errors.iloc[:,3])

#concatenate dummy variables
errors_concat = pd.concat([errors, errors_compensation_type], axis=1)

#Group by using 'subscription_id' and creating new features
errors_groupby = errors_concat.groupby('subscription_id').agg({'reported_date' : ['min','max'], 
                                                               'compensation_amount' : ['sum','max','min','mean','size'], 
                                                               'credit':['sum'], 'full_refund':['sum'], 'none':['sum'], 
                                                               'partial_refund':['sum'], 'refund':['sum'], 
                                                               'sorry':['sum']}).reset_index()

#Column renaming
errors_groupby.columns = ['subscription_id','first_reported_date','last_reported_date','total_CA', 'max_CA','min_CA','mean_CA','no_of_conflict', 
                          'credit', 'full_refund','none','partial_refund','refund','sorry' ]

In [9]:
errors_groupby.head()

,subscription_id,first_reported_date,last_reported_date,total_CA,max_CA,min_CA,mean_CA,no_of_conflict,credit,full_refund,none,partial_refund,refund,sorry
0,1000001,2014-12-05,2014-12-05,10.0,10.0,10.0,10.000000,1,1,0,0,0,0,0
1,1000010,2015-04-02,2016-02-17,119.0,69.0,25.0,39.666667,3,2,1,0,0,0,0
2,1000012,2015-03-08,2015-05-28,138.0,34.5,34.5,34.500000,4,4,0,0,0,0,0
3,1000018,2015-10-20,2015-10-20,34.5,34.5,34.5,34.500000,1,1,0,0,0,0,0
4,1000022,2017-02-07,2017-02-07,69.0,69.0,69.0,69.000000,1,1,0,0,0,0,0


** Pickle file for Errors** 

In [10]:
#Create pickle file for Errors datset
with open('errors.pickle', 'wb') as f:
    pickle.dump(errors_groupby, f)

# Boxes Table

** Data Explration **
* This table has 5 columns named **'subscription_id'**,	**'delivery_date'**',	**'started_week'**,	**'product'**,	**'channel'**
* **'delivery_date'** is given as string which needs to be converted to Timestamp 


** Data Cleaning **
* **'started_week'** could be converted to timestamp to get one more features..

** Feature Engineering **
* Duration between the first and last delivery date for each subscriber will be calculated.
* Total no of delivery for each subscriber is calculated.
* Convert the **channel** and **product** to numerical features using one-hot encoding scheme. It is  creating dummy variables for all the classes of categorical featues.



** Feature Insights **
* **First_delivery_date**: When Pause occured for the first time.
* **Last_delivery_date**: When Pause occured for the last time.
* **No_of_boxes**: No of boxes delivered to the customer.
* **type1, type2,.. **: Dummy variables for **channel**
* **Product1, Product2, .. **: Dummy variables for **product**


In [11]:
boxes.head()

,subscription_id,box_id,delivery_date,started_week,product,channel
0,1000001.0,US243134,2014-11-10,2012-W36,type2,channel16
1,1000001.0,US287248,2014-12-08,2012-W36,type2,channel16
2,1000001.0,US369486,2015-01-19,2012-W36,type2,channel16
3,1000001.0,US299749,2014-12-15,2012-W36,type2,channel16
4,1000001.0,US254024,2014-11-17,2012-W36,type2,channel16


In [ ]:
## Note: Week Handling
## Check all the null columns
## There are few null in Week Column. Need to correct that.
boxes['started_week'][boxes['subscription_id']==1654222.0] = '2017-W09'

#Splitting 'started_week' into year and week to get Timestamp.
boxes['year'], boxes['week'] = boxes['started_week'].str.split('-', 1).str
#Extract Week
boxes['week'] = boxes['week'].str[1:]
#Convert to int type
boxes['week'] = boxes['week'].apply(int)
boxes['year'] = boxes['year'].apply(int)

In [19]:
boxes.head()

,subscription_id,box_id,delivery_date,started_week,product,channel,year,week
0,1000001.0,US243134,2014-11-10,2012-W36,type2,channel16,2012,36
1,1000001.0,US287248,2014-12-08,2012-W36,type2,channel16,2012,36
2,1000001.0,US369486,2015-01-19,2012-W36,type2,channel16,2012,36
3,1000001.0,US299749,2014-12-15,2012-W36,type2,channel16,2012,36
4,1000001.0,US254024,2014-11-17,2012-W36,type2,channel16,2012,36


In [7]:
#Convert to timestamp
boxes.iloc[:,2] = pd.to_datetime(boxes.iloc[:,2], format='%Y-%m-%d')

#Group by over 'subscription_id', 'product' and 'channel'.
boxes_groupby = boxes.groupby(['subscription_id', 'started_week', 'product', 'channel']).agg({'delivery_date':['min', 'max', 'size']}).reset_index()

#Rename columns
boxes_groupby.columns = ['subscription_id', 'started_week', 'Product', 'Channel' ,'First_delivery_date', 'Last_delivery_date', 'No_of_boxes']

#Create dummy column for cATEGORICAL variables 'product' and 'channel' 
boxes_product = pd.get_dummies(boxes_groupby.iloc[:,2])
boxes_channel = pd.get_dummies(boxes_groupby.iloc[:,3])

# Concatenate dummy variables of product and channel
boxes_product = pd.concat([boxes_groupby, boxes_product], axis=1)
boxes_concat = pd.concat([boxes_product, boxes_channel], axis=1)




In [28]:
boxes_concat.head()

,subscription_id,started_week,Product,Channel,First_delivery_date,Last_delivery_date,No_of_boxes,type1,type2,type3,...,channel33,channel34,channel35,channel36,channel4,channel5,channel6,channel7,channel8,channel9
0,1000001.0,2012-W36,type2,channel16,2014-11-10,2015-01-19,6,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1000002.0,2012-W39,type2,channel23,2013-01-14,2013-01-21,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1000003.0,2012-W41,type2,channel23,2013-04-01,2013-04-01,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1000004.0,2012-W41,type2,channel16,2013-09-25,2013-10-23,4,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1000004.0,2012-W41,type2,channel23,2013-01-14,2013-09-09,21,0,1,0,...,0,0,0,0,0,0,0,0,0,0


** Pickle file for Boxes** 

In [35]:
#Create pickle file for Boxes datset
with open('boxes.pickle', 'wb') as f:
    pickle.dump(boxes_concat, f)

# Cancels Tables

** Data Explration **
* This table has 5 columns named **'subscription_id'**,	**'event_type'**',	**'event_date'**.
* **'event_date'** is given as string which needs to be converted to Timestamp 

** Data Cleaning **
* There are approx 2lacs of duplicate rows present in this datset. Remove them.

** Feature Engineering **
* Duration between the first event date and cancellation date for each subscriber will be calculated.
* Total no of cancellation in the total subscription period for each subscriber is calculated.
* Convert the **event_type** to numerical features using one-hot encoding scheme. It is  creating dummy variables for all the values of **event_type**

** Feature Insights **
* **First_event_date**: When reactivation/cancellation occured for the first time.
* **Last_event_date**: When cancellation occured for the last time.
* **Total_cancellation**: Total no of times subscriber cancelled the subscription before final cancellation.
* **Total_reactivation**: Total no of times subscriber reactivate the subscription before final cancellation.



In [36]:
cancels.head()

,subscription_id,event_type,event_date
0,1000001,cancellation,2015-02-25
1,1000008,reactivation,2017-01-07
2,1000008,cancellation,2016-12-10
3,1000008,reactivation,2016-11-29
4,1000008,reactivation,2017-11-03


In [38]:
#Remove all the duplicate rows. It will remove approx 2 Lacs rows.
#Initial rows:1408824 entries #Later rows:1269550 entries
cancels = cancels.drop_duplicates()

In [40]:
#Convert to timestamp
cancels.iloc[:,2] = pd.to_datetime(cancels.iloc[:,2], format='%Y-%m-%d')

#Create dummy column for CATEGORICAL variables 'event_type'.
cancels_event_type = pd.get_dummies(cancels.iloc[:,1])

# Concatenate dummy variables of event_type
cancels = pd.concat([cancels, cancels_event_type], axis=1)
    
#Group by over 'subscription_id'.
cancels_groupby = cancels.groupby('subscription_id').agg({'event_date':['min', 'max'], 
                                                          'cancellation':['sum'], 'reactivation':['sum']}).reset_index()

#Rename columns
cancels_groupby.columns = ['subscription_id', 'First_event_date', 'Last_event_date' ,'Total_cancellation','Total_reactivation']


In [41]:
cancels_groupby.head()

,subscription_id,First_event_date,Last_event_date,Total_cancellation,Total_reactivation
0,1000001,2015-02-25,2015-02-25,1,0
1,1000008,2013-07-08,2017-11-04,4,3
2,1000010,2015-04-02,2016-05-07,2,1
3,1000012,2015-10-24,2016-03-12,2,1
4,1000018,2015-06-07,2016-03-12,4,4


** Pickle file for Cancels** 

In [42]:
#Create pickle file for Cancels datset
with open('cancels.pickle', 'wb') as f:
    pickle.dump(cancels_groupby, f)